In [28]:
import statsmodels.sandbox.distributions.extras as extras
import scipy.interpolate as interpolate
import numpy as np

In [29]:
from statistics import mean
from statistics import variance
from scipy.stats import skew
from scipy.stats import kurtosis

In [30]:
def raw_moment(vec, r):
    return 1 / len(vec) * sum(list(map(lambda i: i**r, vec)))
    
def kelly(retvec, int=6):
    moms = []
    for i in range(int):
        moms.append(raw_moment(retvec, i+1))
    p = []
    for i in range(len(moms)):
        p.append((-1)**i * moms[i])
    r = np.polynomial.polynomial.polyroots(p)
    rr = r.real[abs(r.imag)<1e-5]
    return min(rr)

In [9]:
# https://medium.com/@josh_lxdx/kelly-fraction-f-higher-moments-9eedb0adb22c

def kelly_four_moments(mu, sigma, c3, c4):
    return (c3/(3*c4) - (c3**2/c4**2 - 3*(mu**2 + sigma**2)/c4)/(3*(-c3**3/c4**3 + 9*c3*(mu**2 + sigma**2)/(2*c4**2) + np.sqrt(-4*(c3**2/c4**2 - 3*(mu**2 + sigma**2)/c4)**3 
         + (-2*c3**3/c4**3 + 9*c3*(mu**2 + sigma**2)/c4**2 - 27*mu/c4)**2)/2 - 27*mu/(2*c4))**(1/3))
         - (-c3**3/c4**3 + 9*c3*(mu**2 + sigma**2)/(2*c4**2) + np.sqrt(-4*(c3**2/c4**2 - 3*(mu**2 + sigma**2)/c4)**3
         + (-2*c3**3/c4**3 + 9*c3*(mu**2 + sigma**2)/c4**2 - 27*mu/c4)**2)/2 - 27*mu/(2*c4))**(1/3)/3)

In [10]:
# see https://medium.com/@josh_lxdx/generating-random-variables-with-higher-moments-1bcbd83ec4c8

def generate_normal_four_moments(mu, sigma, skew, kurt, size=1000, sd_wide = 10):
    f = extras.pdf_mvsk([mu, sigma, skew, kurt])
    x = np.linspace(mu - sd_wide * sigma, mu + sd_wide * sigma, num=500)
    y = [f(i) for i in x]
    yy = np.cumsum(y) / np.sum(y)
    inv_cdf = interpolate.interp1d(yy, x, fill_value="extrapolate")
    rr = np.random.rand(size)
    return inv_cdf(rr)

In [42]:
sample = generate_normal_four_moments(.01, .03, .001, 1)

In [43]:
import statistics as stat

In [44]:
_mu = mean(sample)
_sigma = variance(sample)
_skew = skew(sample)
_kurtosis = kurtosis(sample)

In [45]:
# raw moment kelly
kelly(sample,4) 

0.50810388162818

In [46]:
# central moment kelly
kelly_four_moments(_mu, _sigma, _skew, _kurtosis)

-0.24342540849939354

In [48]:
# normal approximation
mean(sample) / variance(sample)

0.5080763525200168